In [1]:
from Scripts import loading as dl


print('######## start data load')
df = dl.load_merged_data('../Data/schema_phone.pkl', '../Data/amazon_phone.pkl', 100, 100)
print('######## finished data load')


######## start data load
######## finished data load


In [41]:
# ToDo: set the values to limit = 5000, schema_size = 0, schema_size <= 1000
limit= 5000
schema_size = 0
amazon_size = limit - schema_size
domains = ['phone', 'movie']
for d in domains:
    amazon_link = f'../Data/amazon_{d}.pkl'
    schema_link = f'../Data/schema_{d}.pkl'

    while schema_size < 1000:
        print(schema_link, amazon_link, schema_size, amazon_size) #run everything
        schema_size = schema_size + 100
        amazon_size = limit - schema_size
    limit = 5000
    schema_size = 0
    amazon_size = limit - schema_size





../Data/schema_phone.pkl ../Data/amazon_phone.pkl 0 5000
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 100 4900
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 200 4800
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 300 4700
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 400 4600
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 500 4500
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 600 4400
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 700 4300
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 800 4200
../Data/schema_phone.pkl ../Data/amazon_phone.pkl 900 4100
../Data/schema_movie.pkl ../Data/amazon_movie.pkl 0 5000
../Data/schema_movie.pkl ../Data/amazon_movie.pkl 100 4900
../Data/schema_movie.pkl ../Data/amazon_movie.pkl 200 4800
../Data/schema_movie.pkl ../Data/amazon_movie.pkl 300 4700
../Data/schema_movie.pkl ../Data/amazon_movie.pkl 400 4600
../Data/schema_movie.pkl ../Data/amazon_movie.pkl 500 4500
../Data/schema_movie.pkl ../Data/amazon_movie.pkl 600 4400
.